In [1]:
import math, os, random, json, pickle, sys, pdb
import string, shutil, time, argparse
import numpy as np
import itertools

from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm as tqdm
from PIL import Image

import torch.nn.functional as F
import torchvision.models as models
import torch, torchvision
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim
import torch.backends.cudnn as cudnn

from torchvision.utils import save_image
from torch.utils.data import DataLoader

from data_loader import ImSituVerbGender
from model import VerbClassification
from logger import Logger

from PIL import ImageFilter

import torch.utils.data as data
import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
class Args:
    annotation_dir='./data'
    balanced=False
    batch_balanced=False
    batch_size=32
    blackout=False
    blackout_box=False
    blackout_face=False
    blur=False
    crop_size=224
    edges=False
    finetune=True
    gender_balanced=False
    grayscale=False
    image_dir='./data/of500_images_resized'
    image_size=256
    learning_rate=0.0001
    no_image=False
    num_epochs=100
    num_verb=211
    ratio="1"
    resume=False
    seed=1
    start_epoch=1
    exp_name = "Baseline_original"
    save_dir = ""
    log_dir = ""
args = Args()

In [3]:
def save_checkpoint(args, state, is_best, filename):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(args.save_dir, 'model_best.pth.tar'))


def train(args, epoch, model, criterion, train_loader, optimizer, \
    train_logger, logging=True):
    model.train()
    nProcessed = 0
    nTrain = len(train_loader.dataset) # number of images
    loss_logger = AverageMeter()

    res = list()
    t = tqdm(train_loader, desc = 'Train %d' % epoch)
    for batch_idx, (images, targets, genders, image_ids) in enumerate(t):
        #if batch_idx == 100: break # constrain epoch size

        if args.batch_balanced:
            man_idx = genders[:, 0].nonzero().squeeze()
            if len(man_idx.size()) == 0: man_idx = man_idx.unsqueeze(0)
            woman_idx = genders[:, 1].nonzero().squeeze()
            if len(woman_idx.size()) == 0: woman_idx = woman_idx.unsqueeze(0)
            selected_num = min(len(man_idx), len(woman_idx))

            if selected_num < args.batch_size / 2:
                continue # skip the batch if the selected num is too small
            else:
                selected_num = args.batch_size / 2
                selected_idx = torch.cat((man_idx[:selected_num], woman_idx[:selected_num]), 0)

            images = torch.index_select(images, 0, selected_idx)
            targets = torch.index_select(targets, 0, selected_idx)
            genders = torch.index_select(genders, 0, selected_idx)

        # set mini-batch dataset
        images = images.cuda()
        targets = targets.cuda()

        # forward, Backward and Optimize
        preds = model(images)

        # compute loss and add softmax to preds (crossentropy loss integrates softmax)
        loss = criterion(preds, targets.max(1, keepdim=False)[1])
        loss_logger.update(loss.item())

        preds = F.softmax(preds, dim=1)
        preds_max = preds.max(1, keepdim=True)[1]

        # save the exact preds (binary)
        tensor = torch.tensor((), dtype=torch.float64)
        preds_exact = tensor.new_zeros(preds.size())
        for idx, item in enumerate(preds_max):
        	preds_exact[idx, item] = 1

        res.append((image_ids, preds.detach().cpu(), targets.detach().cpu(), genders, preds_exact))

        # backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print log info
        nProcessed += len(images)
        t.set_postfix(loss = loss_logger.avg, completed = nProcessed)


    # compute mean average precision score for verb classifier
    total_preds   = torch.cat([entry[1] for entry in res], 0)
    total_targets = torch.cat([entry[2] for entry in res], 0)
    total_genders = torch.cat([entry[3] for entry in res], 0)
    total_preds_exact = torch.cat([entry[4] for entry in res], 0)

    # compute f1 score (no threshold as we simple take the max for multi-classification)
    task_f1_score = f1_score(total_targets.numpy(), total_preds_exact.numpy(), average = 'macro')

    man_idx = total_genders[:, 0].nonzero().squeeze()
    woman_idx = total_genders[:, 1].nonzero().squeeze()

    preds_man = torch.index_select(total_preds, 0, man_idx)
    preds_woman = torch.index_select(total_preds, 0, woman_idx)
    targets_man = torch.index_select(total_targets, 0, man_idx)
    targets_woman = torch.index_select(total_targets, 0, woman_idx)

    meanAP = average_precision_score(total_targets.numpy(), total_preds.numpy(), average='macro')
    meanAP_man = average_precision_score(targets_man.numpy(), preds_man.numpy(), average='macro')
    meanAP_woman = average_precision_score(targets_woman.numpy(), preds_woman.numpy(), average='macro')

    if logging:
        train_logger.scalar_summary('loss', loss_logger.avg, epoch)
        train_logger.scalar_summary('task_f1_score', task_f1_score, epoch)
        train_logger.scalar_summary('meanAP', meanAP, epoch)
        train_logger.scalar_summary('meanAP_man', meanAP_man, epoch)
        train_logger.scalar_summary('meanAP_woman', meanAP_woman, epoch)

    print('man size: {} woman size: {}'.format(len(man_idx), len(woman_idx)))
    print('Train epoch  : {}, meanAP: {:.2f}, meanAP_man: {:.2f}, meanAP_woman: {:.2f}'.format( \
        epoch, meanAP*100, meanAP_man*100, meanAP_woman*100))

def test(args, epoch, model, criterion, val_loader, val_logger, logging=True):

    # set the eval mode
    model.eval()
    nProcessed = 0
    nVal = len(val_loader.dataset) # number of images
    loss_logger = AverageMeter()

    res = list()
    t = tqdm(val_loader, desc = 'Val %d' % epoch)
    for batch_idx, (images, targets, genders, image_ids) in enumerate(t):
        #if batch_idx == 100: break # constrain epoch size

        # Set mini-batch dataset
        images = images.cuda()
        targets = targets.cuda()

        # Forward, Backward and Optimize
        preds = model(images)

        loss = criterion(preds, targets.max(1, keepdim=False)[1])
        loss_logger.update(loss.item())

        preds = F.softmax(preds, dim=1)
        preds_max = preds.max(1, keepdim=True)[1]

        tensor = torch.tensor((), dtype=torch.float64)
        preds_exact = tensor.new_zeros(preds.size())
        for idx, item in enumerate(preds_max):
            preds_exact[idx, item] = 1

        res.append((image_ids, preds.detach().cpu(), targets.detach().cpu(), genders, preds_exact))


        # Print log info
        nProcessed += len(images)
        t.set_postfix(loss = loss_logger.avg, completed = nProcessed)

    # compute mean average precision score for verb classifier
    total_preds   = torch.cat([entry[1] for entry in res], 0)
    total_targets = torch.cat([entry[2] for entry in res], 0)
    total_genders = torch.cat([entry[3] for entry in res], 0)
    total_preds_exact = torch.cat([entry[4] for entry in res], 0)

    task_f1_score = f1_score(total_targets.numpy(), total_preds_exact.numpy(), average = 'macro')

    man_idx = total_genders[:, 0].nonzero().squeeze()
    woman_idx = total_genders[:, 1].nonzero().squeeze()

    preds_man = torch.index_select(total_preds, 0, man_idx)
    preds_woman = torch.index_select(total_preds, 0, woman_idx)
    targets_man = torch.index_select(total_targets, 0, man_idx)
    targets_woman = torch.index_select(total_targets, 0, woman_idx)

    meanAP = average_precision_score(total_targets.numpy(), total_preds.numpy(), average='macro')
    meanAP_man = average_precision_score(targets_man.numpy(), preds_man.numpy(), average='macro')
    meanAP_woman = average_precision_score(targets_woman.numpy(), preds_woman.numpy(), average='macro')

    if logging:
        val_logger.scalar_summary('loss', loss_logger.avg, epoch)
        val_logger.scalar_summary('task_f1_score', task_f1_score, epoch)
        val_logger.scalar_summary('meanAP', meanAP, epoch)
        val_logger.scalar_summary('meanAP_man', meanAP_man, epoch)
        val_logger.scalar_summary('meanAP_woman', meanAP_woman, epoch)

    print('man size: {} woman size: {}'.format(len(man_idx), len(woman_idx)))
    print('Val epoch  : {}, meanAP: {:.2f}, meanAP_man: {:.2f}, meanAP_woman: {:.2f}'.format( \
        epoch, meanAP*100, meanAP_man*100, meanAP_woman*100))

    return task_f1_score

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
verb_id_map = pickle.load(open('./data/verb_id.map', "rb"))
verb2id = verb_id_map['verb2id']
id2verb = verb_id_map['id2verb']

In [5]:
# create model save directory
save_dir = os.path.join('./models', args.exp_name)
args.save_dir =save_dir
if not os.path.exists(save_dir): os.makedirs(save_dir)

In [6]:
# create log save directory for train and val
log_dir = os.path.join('./logs', args.exp_name)
args.log_dir = log_dir
train_log_dir = os.path.join(log_dir, 'train')
val_log_dir = os.path.join(log_dir, 'val')
if not os.path.exists(train_log_dir): os.makedirs(train_log_dir)
if not os.path.exists(val_log_dir): os.makedirs(val_log_dir)
train_logger = Logger(train_log_dir)
val_logger = Logger(val_log_dir)

In [7]:
# image preprocessing
normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],
    std = [0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.Resize(args.image_size),
    transforms.RandomCrop(args.crop_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize])
val_transform = transforms.Compose([
    transforms.Resize(args.image_size),
    transforms.CenterCrop(args.crop_size),
    transforms.ToTensor(),
    normalize])

In [8]:
test_transform = transforms.Compose([
    transforms.Resize(args.image_size),
    transforms.CenterCrop(args.crop_size),
    transforms.ToTensor(),
    normalize])

In [9]:
# Data samplers.
train_data = ImSituVerbGender(args, annotation_dir = args.annotation_dir,
                              image_dir = args.image_dir, split = 'train', transform = train_transform)

val_data = ImSituVerbGender(args, annotation_dir = args.annotation_dir,
                            image_dir = args.image_dir, split = 'val', transform = val_transform)

ImSituVerbGender dataloader
loading train annotations..........
dataset size: 24301
man size : 14199 and woman size: 10102
ImSituVerbGender dataloader
loading val annotations..........
dataset size: 7730
man size : 4457 and woman size: 3273


In [10]:
test_data = ImSituVerbGender(args, annotation_dir = args.annotation_dir,
                            image_dir = args.image_dir, split = 'test', transform = val_transform)

ImSituVerbGender dataloader
loading test annotations..........
dataset size: 7669
man size : 4517 and woman size: 3152


In [11]:
# Data loaders / batch assemblers.
train_loader = torch.utils.data.DataLoader(train_data, batch_size = args.batch_size,
        shuffle = True, num_workers = 0, pin_memory = True)

val_loader = torch.utils.data.DataLoader(val_data, batch_size = args.batch_size,
        shuffle = False, num_workers = 0, pin_memory = True)

In [12]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size = args.batch_size,
        shuffle = False, num_workers = 0, pin_memory = True)

In [13]:
# build the models
model = VerbClassification(args, args.num_verb).cuda()

Build a VerbClassification Model
Load weights from Resnet18/50 done


In [14]:
# build loss
verb_weights = torch.FloatTensor(train_data.getVerbWeights())
criterion = nn.CrossEntropyLoss(weight=verb_weights, reduction='elementwise_mean').cuda()

In [15]:
# build optimizer for trainable loss
def trainable_params():
    for param in model.parameters():
        if param.requires_grad:
            yield param

In [16]:
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('num_trainable_params:', num_trainable_params)
optimizer = torch.optim.Adam(trainable_params(), args.learning_rate, weight_decay = 1e-5)

num_trainable_params: 25989371


In [17]:
best_performance = 0

In [18]:
print('before training, evaluate the model')
test(args, 0, model, criterion, val_loader, val_logger, logging=False)

before training, evaluate the model



man size: 4457 woman size: 3273
Val epoch  : 0, meanAP: 0.71, meanAP_man: 0.78, meanAP_woman: nan


0.0010795686860846764

In [19]:
from tqdm.notebook import trange

In [20]:
for epoch in trange(args.start_epoch, args.num_epochs + 1):
    train(args, epoch, model, criterion, train_loader, optimizer, \
            train_logger, logging = True)
    current_performance = test(args, epoch, model, criterion, val_loader, \
            val_logger, logging = True)
    is_best = current_performance > best_performance
    best_performance = max(current_performance, best_performance)
    model_state = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_performance': best_performance}
    save_checkpoint(args, model_state, is_best, os.path.join(args.save_dir, \
            'checkpoint.pth.tar'))
    # at the end of every run, save the model
    if epoch == args.num_epochs:
        torch.save(model_state, os.path.join(args.save_dir, \
            'checkpoint_%s.pth.tar' % str(args.num_epochs)))


man size: 14199 woman size: 10102
Train epoch  : 1, meanAP: 12.33, meanAP_man: 11.96, meanAP_woman: 11.88



man size: 4457 woman size: 3273
Val epoch  : 1, meanAP: 26.10, meanAP_man: 26.27, meanAP_woman: nan



man size: 14199 woman size: 10102
Train epoch  : 2, meanAP: 26.93, meanAP_man: 25.50, meanAP_woman: 25.54



man size: 4457 woman size: 3273
Val epoch  : 2, meanAP: 30.53, meanAP_man: 30.15, meanAP_woman: nan



man size: 14199 woman size: 10102
Train epoch  : 3, meanAP: 37.25, meanAP_man: 35.77, meanAP_woman: 35.38


...


man size: 14199 woman size: 10102
Train epoch  : 12, meanAP: 87.99, meanAP_man: 87.29, meanAP_woman: 87.41



man size: 4457 woman size: 3273
Val epoch  : 12, meanAP: 35.16, meanAP_man: 34.78, meanAP_woman: nan



man size: 14199 woman size: 10102
Train epoch  : 13, meanAP: 89.74, meanAP_man: 89.08, meanAP_woman: 88.85



man size: 4457 woman size: 3273
Val epoch  : 13, meanAP: 34.77, meanAP_man: 34.06, meanAP_woman: nan


KeyboardInterrupt: 

In [21]:
model_path = os.path.join('./models', args.exp_name)
if os.path.isfile(os.path.join(model_path, 'model_best.pth.tar')):
    print("=> loading encoder from '{}'".format(model_path))
    checkpoint = torch.load(os.path.join(model_path, 'model_best.pth.tar'))
    best_performance = checkpoint['best_performance']
    model.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint (epoch {})".format(checkpoint['epoch']))

=> loading encoder from './models\Baseline_original'
=> loaded checkpoint (epoch 8)


In [22]:
test(args, 0, model, criterion, test_loader, val_logger, logging=False)


man size: 4517 woman size: 3152
Val epoch  : 0, meanAP: 37.21, meanAP_man: nan, meanAP_woman: nan


0.3589980537578331

In [25]:
for param in model.base_network.parameters():
    param.requires_grad = False

In [28]:
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('num_trainable_params:', num_trainable_params)
optimizer_freeze = torch.optim.Adam(trainable_params(), 5e-6, weight_decay = 1e-5)

num_trainable_params: 432339


In [30]:
for epoch in trange(args.start_epoch, args.num_epochs + 1):
    train(args, epoch, model, criterion, train_loader, optimizer_freeze, \
            train_logger, logging = True)
    current_performance = test(args, epoch, model, criterion, val_loader, \
            val_logger, logging = True)
    is_best = current_performance > best_performance
    best_performance = max(current_performance, best_performance)
    model_state = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_performance': best_performance}
    save_checkpoint(args, model_state, is_best, os.path.join(args.save_dir, \
            'checkpoint.pth.tar'))
    # at the end of every run, save the model
    if epoch == args.num_epochs:
        torch.save(model_state, os.path.join(args.save_dir, \
            'checkpoint_%s.pth.tar' % str(args.num_epochs)))


man size: 14199 woman size: 10102
Train epoch  : 1, meanAP: 79.16, meanAP_man: 77.74, meanAP_woman: 78.15



man size: 4457 woman size: 3273
Val epoch  : 1, meanAP: 37.80, meanAP_man: 36.94, meanAP_woman: nan



man size: 14199 woman size: 10102
Train epoch  : 2, meanAP: 79.59, meanAP_man: 77.92, meanAP_woman: 78.65



man size: 4457 woman size: 3273
Val epoch  : 2, meanAP: 37.88, meanAP_man: 37.08, meanAP_woman: nan



man size: 14199 woman size: 10102
Train epoch  : 3, meanAP: 80.11, meanAP_man: 78.79, meanAP_woman: 78.83


...


man size: 14199 woman size: 10102
Train epoch  : 67, meanAP: 83.10, meanAP_man: 81.75, meanAP_woman: 82.07



man size: 4457 woman size: 3273
Val epoch  : 67, meanAP: 38.63, meanAP_man: 37.81, meanAP_woman: nan



man size: 14199 woman size: 10102
Train epoch  : 68, meanAP: 83.20, meanAP_man: 81.92, meanAP_woman: 82.07



man size: 4457 woman size: 3273
Val epoch  : 68, meanAP: 38.62, meanAP_man: 37.77, meanAP_woman: nan


KeyboardInterrupt: 